In [ ]:
%%javascript
$('#header-container').hide();

In [ ]:
from IPython.display import HTML, display

import urllib.parse as urlparse
import ipyvuetify as v
import re

In [ ]:
url = jupyter_notebook_url
query_parameters = urlparse.parse_qs(urlparse.urlparse(url).query)

workbook_id = query_parameters['workbookId'][0] if 'workbookId' in query_parameters else None
condition_id = query_parameters['conditionId'][0] if 'conditionId' in query_parameters else None

if not workbook_id or not condition_id:
    raise Exception('Workbook and Worksheet IDs must be supplied as query parameters in the URL for this Notebook')

# This variable must be set according to the location and name of your Email Notifier notebook.
# Typical installation via the Install script will put all of the notebooks in the same location, 
# so this is assumed for a customization-free installation.
notifier_notebook_url = re.sub(r'\/(notebooks|apps|addon)\/(.*)\/.+$', r'/\1/\2/Email%20Notifier.ipynb', jupyter_notebook_url)

In [ ]:
class Unsubscriber:

    v.theme.themes.light.success = '#007960'
    v.theme.themes.light.primary = '#007960'
    v.theme.themes.light.info = '#2a5c84'

    # The character limit on job indices is too small to use guids - use a shortened guid instead.
    def short_id(self, guid):
        return guid[-12:]
    
    def on_error(self, e):
        message = str(e.message) if hasattr(e, 'message') else str(e)
        
        self.snackbar.color = 'red darken-2'
        self.snackbar.children = ['Something went wrong: ' + message, self.close_snackbar]
        self.snackbar.timeout = 5000
        self.snackbar.v_model = True

    def on_success(self, message):
        self.snackbar.color = 'success'
        self.snackbar.children = [message, self.close_snackbar]
        self.snackbar.timeout = 5000
        self.snackbar.v_model = True
    
    def on_close_snackbar(self, widget, event, data):
        self.snackbar.v_model = False
    
    def on_unsubscribe(self, widget, event, data):
        self.unsubscribe_button.loading = True
        try:
            jobs_df = spy.jobs.pull(datalab_notebook_url=notifier_notebook_url, label=self.workbook_id, all=True)
            if jobs_df is None or jobs_df.empty or self.short_id(self.condition_id) not in jobs_df.index:
                self.unsubscribe_button.loading = False
                self.on_error(RuntimeError(f'This notification is no longer scheduled for Workbook ID' 
                                                  f'{self.workbook_id} and Condition ID {self.condition_id}'))
                return
            referenced_job = jobs_df.loc[self.short_id(self.condition_id)]
            all_address_fields_empty = True
            for address_type in ['To', 'Cc', 'Bcc']:
                if not address_type in referenced_job:
                    continue
                addresses_of_type = referenced_job[address_type].split(',')
                addresses_to_remove = [address.strip() for address in self.email_addresses.v_model.split(',')]
                updated_addresses = [updated_address.strip() for updated_address in addresses_of_type
                                     if updated_address.strip() not in addresses_to_remove]
                address_update = ','.join(updated_addresses)
                if address_update:
                    all_address_fields_empty = False
                jobs_df.loc[self.short_id(self.condition_id), address_type] = address_update

            if all_address_fields_empty:
                jobs_df.drop(index=self.short_id(self.condition_id), inplace=True)
            spy.jobs.push(
                jobs_df,
                datalab_notebook_url=notifier_notebook_url,
                label=self.workbook_id,
                quiet = True
            )
            self.unsubscribe_button.loading = False
            unscheduled_affix = '.  No recipients remain, so the notification job was unscheduled.' \
                if all_address_fields_empty else ''
            self.on_success(f'Unsubscribed{unscheduled_affix}')
        except Exception as e:
            self.unsubscribe_button.loading = False
            self.on_error(e)
            return
            
    
    def __init__(self, workbook_id, condition_id):
        self.workbook_id = workbook_id
        self.condition_id = condition_id
        
        self.close_snackbar = v.Btn(color='white', icon=True, children=[v.Icon(children=['mdi-window-close'])])
        self.snackbar = v.Snackbar(v_model=False, app=True, shaped=True, children=[])
        self.close_snackbar.on_event('click', self.on_close_snackbar)
        
        self.email_addresses = v.Textarea(
            v_model=spy.user.email,
            label='Email Address(es)',
            hint='Separate email addresses with commas',
            rows=1,
            auto_grow=True
        )
        
        self.unsubscribe_button = v.Btn(children=['Unsubscribe'], color='success', class_='mx-2')
        self.unsubscribe_button.on_event('click', self.on_unsubscribe)
        
        self.app = v.App(id='unsubscriber')
        self.appBar = v.AppBar(
            color='white',
            dense=True,
            children=[v.ToolbarTitle(children=['Unsubscribe from Notification'])]
        )
        self.footer = v.Toolbar(children=[self.snackbar, v.Spacer(), self.unsubscribe_button])
        
    def run(self):
        display(HTML("<style>{ max-width:100% !important; } .container { width:100% !important; } .mdi-checkbox-marked, .mdi-minus-box { color: #007960 !important; } .v-toolbar__content { padding: 0 !important } .v-snack {position: absolute !important;top: -220px;right: 0 !important; left: unset !important;} </style>"))
        self.app.children = [self.appBar, self.email_addresses, self.footer]
        return self.app

In [ ]:
unsubscriber = Unsubscriber(workbook_id, condition_id)
unsubscriber.run()